In [6]:
from keras.preprocessing import image
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D
from keras import applications
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers
from keras.models import Sequential
from keras.layers import Dropout, Flatten, Dense
from keras.models import Model
 
# Размер изображений
img_width, img_height = 150, 150
# Путь к каталогу с изображениями для обучения
train_data_dir = 'data/train'
# Путь к каталогу с изображениями для валидации
validation_data_dir = 'data/validation'
# Количество изображений для обучения
nb_train_samples = 2000
# Количество изображений для валидации
nb_validation_samples = 800
# Количество эпох
epochs = 5
# Размер выборки
batch_size = 16
 
# Загружаем сеть VGG16 без части, которая отвечает за классификацию
base_model = applications.VGG16(weights='imagenet', include_top=False)
 
# Добавляем слои классификации
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
# Выходной слой с двумя нейронами для классов "кот" и "собака"
predictions = Dense(2, activation='softmax')(x)
 
# Составляем сеть из двух частей
model = Model(inputs=base_model.input, outputs=predictions)
 
# "Замораживаем" сверточные уровни сети VGG16
# Обучаем только вновь добавленные слои
for layer in base_model.layers:
    layer.trainable = False
 
# Компилируем модель
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])
 
# Создаем генератор данных для обучения
datagen = ImageDataGenerator(rescale=1. / 255)
train_generator = datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode= 'categorical')
 
# Создаем генератор данных для валидации
validation_generator = datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode= 'categorical')
 


Found 20000 images belonging to 2 classes.
Found 5000 images belonging to 2 classes.
Epoch 1/5
1999/2000 [============================>.] - ETA: 1s - loss: 0.2950 - acc: 0.8709

KeyboardInterrupt: 

In [7]:
# Обучаем модель с помощью генератора
model.fit_generator(
    train_generator,
    steps_per_epoch=nb_train_samples,
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps = nb_validation_samples)
 

Epoch 1/5
  27/2000 [..............................] - ETA: 45:25 - loss: 0.2742 - acc: 0.8912

KeyboardInterrupt: 

In [1]:
# Сохраняем нейронную сеть 
model_json = model.to_json()
json_file = open("vgg16_cat_dogs.json", "w")
# Записываем архитектуру сети в файл
json_file.write(model_json)
json_file.close()
# Записываем данные о весах в файл
model.save_weights("vgg16_cat_dogs.h5")
print("Сохранение сети завершено")

NameError: name 'model' is not defined

In [4]:
import numpy as np
from keras.models import model_from_json
from keras.preprocessing import image
from keras.applications.vgg16 import preprocess_input
import matplotlib.pyplot as plt
#from scipy.misc import toimage

# Список классов
classes = ['кот', 'собака']

json_file = open("vgg16_cat_dogs.json", "r")
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
loaded_model.load_weights("vgg16_cat_dogs.h5")

loaded_model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])

img = image.load_img('cat-kot-ryzhiy.jpg', target_size=(224, 224))

x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)
x = preprocess_input(x)

prediction = loaded_model.predict(x)

print(prediction)
print(classes[np.argmax(prediction)])

[[1.0000000e+00 7.0519474e-16]]
кот


In [10]:
# task 2

import dlib
from skimage import io
from scipy.spatial import distance

sp = dlib.shape_predictor('shape_predictor_68_face_landmarks.dat')
facerec = dlib.face_recognition_model_v1('dlib_face_recognition_resnet_model_v1.dat')
detector = dlib.get_frontal_face_detector()

img = io.imread('1.png')
win1 = dlib.image_window()
win1.clear_overlay()
win1.set_image(img[:,:,:3])

dets = detector(img[:,:,:3], 1)
for k, d in enumerate(dets):
    print("Detection {}: Left: {} Top: {} Right: {} Bottom: {}".format(
        k, d.left(), d.top(), d.right(), d.bottom()))
    shape = sp(img[:,:,:3], d)
    win1.clear_overlay()
    win1.add_overlay(d)
    win1.add_overlay(shape)
    
face_descriptor1 = facerec.compute_face_descriptor(img[:,:,:3], shape)

print(face_descriptor1)

img = io.imread('6.png')[:,:,:3]
win2 = dlib.image_window()
win2.clear_overlay()
win2.set_image(img)
dets_webcam = detector(img, 1)
for k, d in enumerate(dets_webcam):
    print("Detection {}: Left: {} Top: {} Right: {} Bottom: {}".format(
        k, d.left(), d.top(), d.right(), d.bottom()))
    shape = sp(img, d)
    win2.clear_overlay()
    win2.add_overlay(d)
    win2.add_overlay(shape)
face_descriptor2 = facerec.compute_face_descriptor(img, shape)

a = distance.euclidean(face_descriptor1, face_descriptor2)
print(a)

Detection 0: Left: 22 Top: 171 Right: 290 Bottom: 439
-0.0508404
0.0987085
-0.0182569
-0.118395
-0.174268
0.0563256
-0.0591454
-0.0794522
0.21135
-0.0717236
0.232684
0.0774581
-0.283534
-0.037678
0.0142333
0.15079
-0.144254
-0.137683
-0.10436
-0.129029
0.00395896
-0.0111611
-0.019151
0.0782404
-0.0661
-0.207409
-0.0137053
-0.131252
0.137101
-0.152323
-0.00557941
0.0152269
-0.0840521
-0.00404333
0.0118073
0.0484435
-0.0781863
-0.111951
0.271781
-0.0394491
-0.161094
0.0151805
0.0208086
0.220519
0.209084
-0.0229043
0.121446
-0.0854881
0.154066
-0.170626
0.022073
0.160177
0.200875
0.0431131
0.00219897
-0.21531
0.00295505
0.144186
-0.251927
0.0704762
0.148305
-0.100726
-0.124627
-0.0638406
0.248889
0.082368
-0.185441
-0.207279
0.167242
-0.170039
-0.12137
0.0981234
-0.147069
-0.133503
-0.216835
0.00718258
0.354778
0.156668
-0.191906
0.0035893
-0.103962
-0.0351274
-0.0399731
0.0499066
-0.102391
-0.0796103
-0.123418
-0.0249588
0.184774
-0.010939
0.0173452
0.174989
0.118418
-0.0155421
-0.049176